In [1]:
import numpy as np
import pandas as pd
from tqdm import tqdm

import re
import string

import nltk
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer  
stop_words = stopwords.words('english')


from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix,classification_report
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import confusion_matrix,classification_report

import matplotlib.pyplot as plt
import seaborn as sns

from datasets import load_dataset
from sklearn.model_selection import KFold
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.preprocessing import LabelEncoder

In [11]:
# Try loading a specific split of the dataset
dataset = load_dataset("google-research-datasets/go_emotions")

# Check the loaded data
print(dataset)

Extracting data files:   0%|          | 0/3 [00:00<?, ?it/s]

Generating train split:   0%|          | 0/211225 [00:00<?, ? examples/s]

DatasetGenerationError: An error occurred while generating the dataset

In [14]:
# Memeriksa label yang tersedia dalam dataset
train_data = dataset['train']
print(train_data.features['label'].names)

['sadness', 'joy', 'love', 'anger', 'fear', 'surprise']


In [19]:
# Memuat dataset dari Hugging Face
dataset = load_dataset("go_emotions")

# Menggabungkan data latih dan validasi
data = dataset['train']
texts = data['text']
labels = data['label']

# TF-IDF vektorizer
max_features = 10000  # Jumlah kata unik yang akan digunakan
tfidf_vectorizer = TfidfVectorizer(max_features=max_features)
x_data = tfidf_vectorizer.fit_transform(texts).toarray()

# Mengubah label menjadi bentuk numerik
label_encoder = LabelEncoder()
y_data = label_encoder.fit_transform(labels)

# K-Fold Cross Validation
kf = KFold(n_splits=5, shuffle=True, random_state=42)
accuracies = []

for train_index, val_index in kf.split(x_data):
    x_train, x_val = x_data[train_index], x_data[val_index]
    y_train, y_val = y_data[train_index], y_data[val_index]
    
    # Membuat model Random Forest
    model = RandomForestClassifier(n_estimators=100, random_state=42)
    model.fit(x_train, y_train)
    
    # Prediksi dan evaluasi
    y_pred = model.predict(x_val)
    accuracy = accuracy_score(y_val, y_pred)
    accuracies.append(accuracy)
    print(f'Fold accuracy: {accuracy:.4f}')

# Rata-rata akurasi
print(f'Mean accuracy: {np.mean(accuracies):.4f}')

Fold accuracy: 0.8634
Fold accuracy: 0.8562
Fold accuracy: 0.8509
Fold accuracy: 0.8556
Fold accuracy: 0.8653
Mean accuracy: 0.8583
